In [0]:
from pyspark.sql.functions import *

In [0]:
%sql
USE SCHEMA federal_spending_bronze

## Load from Bronze Table

In [0]:
spending24_df = spark.table('2024_federal_spending_bronze_table')

## Drop Unneeded Columns

In [0]:
spending24_df = spending24_df.drop('reporting_agency_name','federal_account_symbol','spending_authority_from_offsetting_collections_amount','borrowing_authority_amount','contract_authority_amount','agency_identifier_name','budget_authority_unobligated_balance_brought_forward','adjustments_to_unobligated_balance_brought_forward_cpe','budget_authority_appropriated_amount','total_other_budgetary_resources_amount')

## Rename Columns

In [0]:
spending24_df = spending24_df \
    .withColumnRenamed('last_modified_date','agency_name') \
    .withColumnRenamed('federal_account_name','account_name')

## Load to Silver Layer

In [0]:
spending24_df.createOrReplaceTempView('2024_federal_spending_silver_table') # Create Temp view for loading via SQL

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.federal_spending_silver.2024_federal_spending_silver_table AS
SELECT * FROM 2024_federal_spending_silver_table


## Aggregate Data 

In [0]:
%sql
USE SCHEMA federal_spending_silver

In [0]:
%sql
SELECT * FROM 2024_federal_spending_silver_table

In [0]:
%sql
    Create Table workspace.federal_spending_gold.2024_federal_spending_by_function as 
    SELECT
      budget_function,
      round(sum(total_budgetary_resources)) as 2024_resources,
      round(sum(obligations_incurred)) as 2024_obligations,
      round(sum(unobligated_balance)) as 2024_unobligated_balance,
      round(SUM(obligations_incurred) / SUM(SUM(obligations_incurred)) OVER () * 100,2) as percent_of_obligations,
      round(SUM(total_budgetary_resources) / SUM(SUM(total_budgetary_resources)) OVER () * 100,2) as percent_of_resources
      
    FROM 
      2024_federal_spending_silver_table
    GROUP BY 
      budget_function
    ORDER BY 2024_resources DESC
 